## Notebook for Smillie data integration and batch correction `scVI`

- **Developed by**: Anna Maguza
- **Institute of Computational Biology - Computational Health Centre - Helmholtz Munich**
- 4th July 2023

### Load required modules

In [1]:
import sys
import scvi
import torch
import anndata
import matplotlib
import matplotlib.pyplot as plt
from matplotlib.colors import LinearSegmentedColormap
import seaborn as sns

import numpy as np
import scipy as sp
import pandas as pd
import scanpy as sc
import numpy.random as random


from umap import UMAP
import warnings; warnings.simplefilter('ignore')

import matplotlib.pyplot as plt

Global seed set to 0
/Users/anna.maguza/miniconda3/envs/scvi_env/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/Users/anna.maguza/miniconda3/envs/scvi_env/lib/python3.9/site-packages/flax/struct.py:132: FutureWarning: jax.tree_util.register_keypaths is deprecated, and will be removed in a future release. Please use `register_pytree_with_keys()` instead.
  jax.tree_util.register_keypaths(data_clz, keypaths)
/Users/anna.maguza/miniconda3/envs/scvi_env/lib/python3.9/site-packages/flax/struct.py:132: FutureWarning: jax.tree_util.register_keypaths is deprecated, and will be removed in a future release. Please use `register_pytree_with_keys()` instead.
  jax.tree_util.register_keypaths(data_clz, keypaths)
/Users/anna.maguza/miniconda3/envs/scvi_env/lib/python3.9/site-packages/flax/struct.py:132: FutureWarnin

In [2]:
%matplotlib inline
matplotlib.rcParams["pdf.fonttype"] = 42
matplotlib.rcParams["ps.fonttype"] = 42

In [3]:
torch.cuda.is_available()

False

In [4]:
torch.set_float32_matmul_precision('medium')

In [5]:
sc.settings.verbosity = 3
sc.logging.print_versions()
sc.settings.set_figure_params(dpi = 180, color_map = 'magma_r', dpi_save = 300, vector_friendly = True, format = 'svg')

-----
anndata     0.8.0
scanpy      1.9.3
-----
PIL                         9.4.0
absl                        NA
appnope                     0.1.2
asttokens                   NA
attr                        22.2.0
backcall                    0.2.0
beta_ufunc                  NA
binom_ufunc                 NA
brotli                      NA
certifi                     2022.12.07
cffi                        1.15.1
charset_normalizer          2.1.1
chex                        0.1.6
colorama                    0.4.6
comm                        0.1.2
contextlib2                 NA
cycler                      0.10.0
cython_runtime              NA
dateutil                    2.8.2
debugpy                     1.5.1
decorator                   5.1.1
defusedxml                  0.7.1
docrep                      0.3.2
entrypoints                 0.4
executing                   0.8.3
flax                        0.6.1
fsspec                      2023.3.0
h5py                        3.8.0
hypergeom_uf

In [6]:
arches_params = dict(
    use_layer_norm = "both",
    use_batch_norm = "none",
    encode_covariates = True,
    dropout_rate = 0.2,
    n_layers = 2,
)

In [7]:
def X_is_raw(adata):
    return np.array_equal(adata.X.sum(axis=0).astype(int), adata.X.sum(axis=0))

### Read in datasets

In [8]:
input = '/Users/anna.maguza/Desktop/Data/Gut_project/Kong_2023/Raw_anndata/adata_Kong_2023_healthy_with_QC.h5ad'
adata = sc.read_h5ad(input)

In [9]:
input = '/Users/anna.maguza/Desktop/Data/Gut_project/Kong_2023/Raw_anndata/Kong_2023_raw_anndata.h5ad'
adata_raw = sc.read_h5ad(input)

In [10]:
# Filter out cells from adata_raw that are not in adata
adata_raw = adata_raw[adata.obs_names]

# Replace counts in adata to raw counts from adata_raw
adata.X = adata_raw.X

del adata_raw

In [11]:
X_is_raw(adata)

True

In [12]:
# Save raw data
adata.raw = adata

In [15]:
adata.obs_keys

<bound method AnnData.obs_keys of AnnData object with n_obs × n_vars = 181806 × 27830
    obs: 'cell_type', 'tissue', 'batch', 'biosample_id', 'n_genes', 'n_counts', 'Chem', 'Site', 'Type', 'donor_id', 'Layer', 'Celltype', 'sex', 'species', 'species__ontology_label', 'library_preparation_protocol', 'library_preparation_protocol__ontology_label', 'organ', 'organ__ontology_label', 'disease', 'disease__ontology_label', 'n_genes_by_counts', 'log1p_n_genes_by_counts', 'total_counts', 'log1p_total_counts', 'pct_counts_in_top_50_genes', 'pct_counts_in_top_100_genes', 'pct_counts_in_top_200_genes', 'pct_counts_in_top_500_genes', 'total_counts_mito', 'pct_counts_mito', 'total_counts_ribo', 'pct_counts_ribo', 'S_score', 'G2M_score', 'doublet_scores', 'predicted_doublets', 'doublet_info', 'percent_chrY', 'XIST-counts'
    var: 'gene_id', 'gene_name', 'n_cells_by_counts', 'mean_counts', 'log1p_mean_counts', 'pct_dropout_by_counts', 'total_counts', 'log1p_total_counts', 'mito', 'ribo', 'highly_vari

In [18]:
adata.obs['cell_type'].value_counts()

Immune        81599
Epithelial    77638
Stromal       22569
Name: cell_type, dtype: int64

In [17]:
adata.layers['counts'] = adata.X.copy()

# Calculate 5000 HVGs
sc.pp.highly_variable_genes(
    adata,
    flavor = "seurat_v3",
    n_top_genes = 5000,
    layer = "counts",
    batch_key = 'library_preparation_protocol__ontology_label',
    subset = True,
    span = 1
)

If you pass `n_top_genes`, all cutoffs are ignored.
extracting highly variable genes
--> added
    'highly_variable', boolean vector (adata.var)
    'highly_variable_rank', float vector (adata.var)
    'means', float vector (adata.var)
    'variances', float vector (adata.var)
    'variances_norm', float vector (adata.var)


In [20]:
adata.obs.rename(columns = {'Celltype': 'Cell States'}, inplace = True)
adata.obs.rename(columns = {'cell_type': 'Cell Type'}, inplace = True)

In [21]:
# List of specific Cell States to look for
cell_states_list = [
    'Stem cells OLFM4 LGR5',
    'Stem cells OLFM4 PCNA',
    'Stem cells OLFM4 GSTA1',
    'Stem cells OLFM4'
]

# Add 'Stem Cell' as a new category to the 'Cell Type' column
adata.obs['Cell Type'] = adata.obs['Cell Type'].cat.add_categories(['Stem Cell'])

# Update 'Cell Type' based on the condition in 'Cell States'
adata.obs.loc[adata.obs['Cell States'].isin(cell_states_list), 'Cell Type'] = 'Stem Cell'

In [22]:
# List of specific Cell States to look for
cell_states_list = [
    'B cells',
    'B cells AICDA LRMP'
]

# Add 'B cells' as a new category to the 'Cell Type' column
adata.obs['Cell Type'] = adata.obs['Cell Type'].cat.add_categories(['B cells'])

# Update 'Cell Type' based on the condition in 'Cell States'
adata.obs.loc[adata.obs['Cell States'].isin(cell_states_list), 'Cell Type'] = 'B cells'

In [23]:
# List of specific Cell States to look for
cell_states_list = [
    'T cells CD4 FOSB',
    'T cells CD4 IL17A',
    'T cells CD8',
    'T cells CD8 KLRG1',
    'T cells Naive CD4',
    'T cells OGT',
    'Tregs',
    'NK cells KLRF1 CD3G-',
    'NK-like cells ID3 ENTPD1',
    'ILCs',
    'IELs ID3 ENTPD1',
    'Lymphatics'
]

# Add 'T cells' as a new category to the 'Cell Type' column
adata.obs['Cell Type'] = adata.obs['Cell Type'].cat.add_categories(['T cells'])

# Update 'Cell Type' based on the condition in 'Cell States'
adata.obs.loc[adata.obs['Cell States'].isin(cell_states_list), 'Cell Type'] = 'T cells'

In [24]:
cell_states_list = [
    'Plasma cells'
]

# Add 'Plasma cells' as a new category to the 'Cell Type' column
adata.obs['Cell Type'] = adata.obs['Cell Type'].cat.add_categories(['Plasma cells'])

# Update 'Cell Type' based on the condition in 'Cell States'
adata.obs.loc[adata.obs['Cell States'].isin(cell_states_list), 'Cell Type'] = 'Plasma cells'

In [25]:
cell_states_list = [
    'Cycling cells',
    'DC1',
    'DC2 CD1D',
    'DC2 CD1D-',
    'Immune Cycling cells',
    'Macrophages',
    'Macrophages CCL3 CCL4',
    'Macrophages CXCL9 CXCL10',
    'Macrophages LYVE1',
    'Macrophages Metallothionein',
    'Macrophages PLA2G2D',
    'Mast cells',
    'Mature DCs',
    'Monocytes CHI3L1 CYP27A1', 
    'Monocytes HBB',
    'Monocytes S100A8 S100A9'
]

# Add 'Myeloid' as a new category to the 'Cell Type' column
adata.obs['Cell Type'] = adata.obs['Cell Type'].cat.add_categories(['Myeloid'])

# Update 'Cell Type' based on the condition in 'Cell States'
adata.obs.loc[adata.obs['Cell States'].isin(cell_states_list), 'Cell Type'] = 'Myeloid'

In [27]:
cell_states_list = [
    'Glial cells'
]

# Add 'Neuronal' as a new category to the 'Cell Type' column
adata.obs['Cell Type'] = adata.obs['Cell Type'].cat.add_categories(['Neuronal'])

# Update 'Cell Type' based on the condition in 'Cell States'
adata.obs.loc[adata.obs['Cell States'].isin(cell_states_list), 'Cell Type'] = 'Neuronal'

In [28]:
cell_states_list = [
    'Endothelial cells CA4 CD36',
    'Endothelial cells CD36',
    'Endothelial cells DARC',
    'Endothelial cells LTC4S SEMA3G'
]

# Add 'Endothelial' as a new category to the 'Cell Type' column
adata.obs['Cell Type'] = adata.obs['Cell Type'].cat.add_categories(['Endothelial'])

# Update 'Cell Type' based on the condition in 'Cell States'
adata.obs.loc[adata.obs['Cell States'].isin(cell_states_list), 'Cell Type'] = 'Endothelial'

In [29]:
cell_states_list = [
    'Activated fibroblasts CCL19 ADAMADEC1',
    'Fibroblasts ADAMDEC1',
    'Fibroblasts KCNN3 LY6H',
    'Fibroblasts NPY SLITRK6',
    'Fibroblasts SFRP2 SLPI',
    'Fibroblasts SMOC2 PTGIS',
    'Inflammatory fibroblasts IL11 CHI3L1',
    'Pericytes HIGD1B STEAP4',
    'Pericytes RERGL NTRK2',
    'Stromal Cycling cells',
    'Myofibroblasts GREM1 GREM2',
    'Myofibroblasts HHIP NPNT'
]

# Add 'Mesenchymal' as a new category to the 'Cell Type' column
adata.obs['Cell Type'] = adata.obs['Cell Type'].cat.add_categories(['Mesenchymal'])

# Update 'Cell Type' based on the condition in 'Cell States'
adata.obs.loc[adata.obs['Cell States'].isin(cell_states_list), 'Cell Type'] = 'Mesenchymal'

In [30]:
adata.obs.rename(columns = {'Cell Type': 'Cell_Type'}, inplace = True)

### Run Integration with scVI

In [32]:
adata = adata.copy()
scvi.model.SCVI.setup_anndata(adata, 
                              layer = "counts", 
                              labels_key = "Cell_Type", 
                              categorical_covariate_keys = ["biosample_id"])

In [33]:
scvi_model = scvi.model.SCVI(adata, n_latent = 50, n_layers = 3, dispersion = 'gene-batch', gene_likelihood = 'nb')

In [34]:
scvi_model.train()

GPU available: True (mps), used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


Epoch 44/44: 100%|██████████| 44/44 [1:18:41<00:00, 98.40s/it, loss=712, v_num=1]   

`Trainer.fit` stopped: `max_epochs=44` reached.


Epoch 44/44: 100%|██████████| 44/44 [1:18:41<00:00, 107.31s/it, loss=712, v_num=1]


In [35]:
adata.obsm["X_scVI"] = scvi_model.get_latent_representation()

### Integration with scANVI

In [36]:
scanvi_model = scvi.model.SCANVI.from_scvi_model(
    scvi_model,
    adata=adata,
    labels_key="Cell_Type",
    unlabeled_category="Unknown",
)

In [37]:
scanvi_model.train()

INFO     Training for 10 epochs.                                                                                   


GPU available: True (mps), used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


Epoch 10/10: 100%|██████████| 10/10 [16:10<00:00, 97.76s/it, loss=789, v_num=1]

`Trainer.fit` stopped: `max_epochs=10` reached.


Epoch 10/10: 100%|██████████| 10/10 [16:10<00:00, 97.06s/it, loss=789, v_num=1]


In [38]:
adata.obsm["X_scANVI"] = scanvi_model.get_latent_representation(adata)

In [39]:
adata.write('/Users/anna.maguza/Desktop/Data/Processed_datasets/1_QC/Kong_scVI_scANVI.h5ad')

### UMAP calculation

In [40]:
sc.pp.neighbors(adata, use_rep = "X_scANVI", n_neighbors = 50, metric = 'minkowski')

computing neighbors
    finished: added to `.uns['neighbors']`
    `.obsp['distances']`, distances for each pair of neighbors
    `.obsp['connectivities']`, weighted adjacency matrix (0:01:01)


In [41]:
sc.tl.umap(adata, min_dist = 0.4, spread = 4, random_state = 1712)

computing UMAP
    finished: added
    'X_umap', UMAP coordinates (adata.obsm) (0:03:04)


In [60]:
adata.write('/Users/anna.maguza/Desktop/Data/Processed_datasets/1_QC/Kong_scVI_scANVI.h5ad')

In [43]:
# Change 'CO' in Kong_adata.obs['Site'] to 'Colon', 'TI' to 'Terminal Ileum', and 'SB' to 'Small Bowel'
adata.obs['Site'] = adata.obs['Site'].replace('CO', 'Colon')
adata.obs['Site'] = adata.obs['Site'].replace('TI', 'Terminal Ileum')
adata.obs['Site'] = adata.obs['Site'].replace('SB', 'Small Bowel')

# Rename columns in Kong-2023 dataset as in GCA_Smillie_Wang dataset
adata.obs.rename(columns = {'donor_id': 'Donor_ID'}, inplace = True)
adata.obs['Study_name'] = 'Kong 2023'
adata.obs.rename(columns = {'biosample_id': 'Sample_ID'}, inplace = True)
adata.obs.rename(columns = {'Site': 'Location'}, inplace = True)
adata.obs.rename(columns = {'Cell States': 'Cell_States'}, inplace = True)

adata.obs.rename(columns = {'library_preparation_protocol__ontology_label': 'Library_Preparation_Protocol'}, inplace = True)

adata.obs.drop(columns = ['log1p_n_genes_by_counts', 'log1p_total_counts', 'log1p_total_counts', 'pct_counts_in_top_50_genes', 'pct_counts_in_top_100_genes', 'pct_counts_in_top_200_genes', 'pct_counts_in_top_500_genes', 'total_counts'], inplace = True)

adata.obs['sex'] = adata.obs['sex'].replace('male', 'Male')
adata.obs['sex'] = adata.obs['sex'].replace('female', 'Female')
adata.obs.rename(columns = {'sex': 'Sex'}, inplace = True)

adata.obs['disease__ontology_label'] = adata.obs['disease__ontology_label'].replace('normal', 'Healthy adult')
adata.obs.rename(columns = {'disease__ontology_label': 'Diagnosis'}, inplace = True)

adata.obs.drop(columns = ['G2M_score', 'percent_chrY', 'doublet_info', 'XIST-counts', 'S_score'], inplace = True)

adata.obs.drop(columns = ['organ', 'tissue', 'Type', 'library_preparation_protocol', 'disease', 'organ__ontology_label', 'species', 'species__ontology_label'], inplace = True)

In [ ]:
sc.set_figure_params(dpi=300)
sc.pl.umap(adata, frameon = False, color = ['Cell_Type', 'Donor_ID', 'Sample_ID', 'Location', 'Sex', 'Cell States', 'Library_Preparation_Protocol'], size = 1, legend_fontsize = 5, ncols = 3)

In [ ]:
sc.set_figure_params(dpi=300)
sc.pl.umap(adata, frameon = False, color = ['Cell_Type', 'Cell States'], size = 1, legend_fontsize = 5, ncols = 3)

In [ ]:
sc.set_figure_params(dpi=300)
sc.pl.umap(adata, frameon = False, color = ['Cell_Type', 'Donor_ID', 'Sample_ID', 'Location', 'Sex', 'Library_Preparation_Protocol'], size = 1, legend_fontsize = 5, ncols = 3)

In [65]:
adata.obs['predicted_doublets'] = adata.obs['predicted_doublets'].astype(str)
adata.obs['n_counts'] = adata.obs['n_counts'].astype(int)
adata.obs['n_genes'] = adata.obs['n_genes'].astype(int)

In [64]:
adata.obs_keys

<bound method AnnData.obs_keys of AnnData object with n_obs × n_vars = 181806 × 5000
    obs: 'Cell_Type', 'batch', 'Sample_ID', 'n_genes', 'n_counts', 'Chem', 'Location', 'Donor_ID', 'Layer', 'Cell States', 'Sex', 'Library_Preparation_Protocol', 'Diagnosis', 'n_genes_by_counts', 'total_counts_mito', 'pct_counts_mito', 'total_counts_ribo', 'pct_counts_ribo', 'doublet_scores', 'predicted_doublets', '_scvi_batch', '_scvi_labels', 'Study_name'
    var: 'gene_id', 'gene_name', 'n_cells_by_counts', 'mean_counts', 'log1p_mean_counts', 'pct_dropout_by_counts', 'total_counts', 'log1p_total_counts', 'mito', 'ribo', 'highly_variable', 'highly_variable_rank', 'means', 'variances', 'variances_norm', 'highly_variable_nbatches'
    uns: 'biosample_id_colors', 'donor_id_colors', 'hvg', 'neighbors', 'umap', '_scvi_uuid', '_scvi_manager_uuid', 'Cell_Type_colors', 'Diagnosis_colors', 'Sample_ID_colors', 'Donor_ID_colors', 'Location_colors', 'Sex_colors', 'Cell States_colors', 'Library_Preparation_Protoc

In [ ]:
sc.set_figure_params(dpi=300)
sc.pl.umap(adata, frameon = False, color = ['n_genes_by_counts', 'n_counts', 'pct_counts_mito', 'pct_counts_ribo', 'predicted_doublets', ], size = 1, legend_fontsize = 5, ncols = 3)

In [79]:
# Make a column 'Stem_cell' in adata.obs, and put True if adata.obs['Cell_State'] == 'Stem_cell', False otherwise
adata.obs['Stem_cell'] = adata.obs['Cell_Type'] == 'Stem Cell'

In [80]:
adata.obs['Stem_cell'] = adata.obs['Stem_cell'].astype(str)

In [ ]:
# Plot only Stem cells
sc.set_figure_params(dpi=300)
sc.pl.umap(adata, frameon = False, color = ['Stem_cell'], size = 1, legend_fontsize = 5, ncols = 3)

In [82]:
# Return to raw counts
adata = adata.raw.to_adata()

In [ ]:
Stem_cells_markers = ['CD24', 'DCLK1', 'LGR5', 'CD166', 'CD44', 'DCAMKL-1', 'SOX9', 'ACAD10', 'ACVR1C', 'ADH1C', 'ALDH1', 'ALK3', 'ARSE', 
'ASCL2', 'ATP10B', 'BMI1', 'C16orf89', 'C6orf136', 'CD29', 'CDCA7', 'CFTR','CHMP4C', 'CHP2', 'CLDN15', 'CLDN18', 'CLDN2', 'CPA6', 'DAPK2', 
'DDC', 'EFNA3', 'EPHB2', 'EPYC', 'EVPL', 'F2RL1', 'FBLN2', 'FOXD2-AS1', 'GATA6-AS1', 'GDF15', 'GJB1', 'GJB1', 'GOLT1A', 'GPX2', 'HNF1A', 
'HSD17B2', 'ITPKC','LEFTY1', 'LHFPL3-AS2', 'LIPG', 'LY6G6D', 'MGST1', 'MSI1', 'MYOM3', 'Musashi-1', 'NOX1', 'OLFM4', 'PCSK9', 'PDZD3', 
'PHLDA1', 'PKP2', 'PLAGL2', 'PLEKHH1', 'PPP1R1B', 'PTGDR', 'PTK7', 'RGMB', 'RNF157', 'RNF186', 'SFN', 'SLC27A2', 'SLC38A4', 'SLPI',
'SULT1B1', 'TAF4B', 'TANC1', 'TMEM171', 'TSPAN8', 'Telomerase Inhibitors', 'URB1-AS1', 'ZBED9', 'ZNF296', 'ASCL2', 'SMOC2']
sc.tl.score_genes(adata, Stem_cells_markers, score_name = 'Stem_cells_markers_score')

sc.set_figure_params(dpi=300)
sc.pl.umap(adata, color= ['Stem_cells_markers_score'], color_map = "RdPu", size = 0.3, frameon = False)